In [54]:
import pandas as pd
import os

In [55]:
DATA_DIR = os.path.join(os.getcwd(), '..', 'data')

In [56]:
df = pd.read_csv(os.path.join(DATA_DIR, "retail_store_inventory.csv"))
df.head()

# Take only certain amount of  data
df = df[:5000]


In [57]:
sorted_df = df.sort_values(['Store ID', 'Product ID'], ascending=[True, True])

In [58]:
prev_s_id = "S001"
count = 0
for idx, row in sorted_df.iterrows():
  s_id = row['Store ID']
  if (s_id == prev_s_id):
    count += 1
    new_p_id ="P"+"{:04d}".format(count)

  else:
    count = 1
    new_p_id ="P"+"{:04d}".format(count)

  sorted_df.at[idx, "Product ID"] = new_p_id
  prev_s_id = s_id

sorted_df.reset_index(drop=True)

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn
1,2022-01-02,S001,P0002,Groceries,West,116,81,104,92.94,27.95,10,Cloudy,0,30.89,Spring
2,2022-01-03,S001,P0003,Electronics,West,154,5,189,5.36,62.70,20,Rainy,0,58.22,Winter
3,2022-01-04,S001,P0004,Groceries,South,85,58,193,52.87,77.88,15,Cloudy,1,75.99,Winter
4,2022-01-05,S001,P0005,Groceries,South,238,147,37,150.27,28.46,20,Sunny,1,29.40,Winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2022-02-15,S005,P0996,Clothing,West,359,114,193,108.98,28.77,5,Sunny,1,25.28,Spring
4996,2022-02-16,S005,P0997,Toys,North,350,143,182,152.28,83.85,20,Rainy,0,87.49,Autumn
4997,2022-02-17,S005,P0998,Toys,North,441,2,149,1.44,24.23,15,Sunny,0,19.71,Autumn
4998,2022-02-18,S005,P0999,Electronics,South,186,67,82,57.23,50.35,5,Cloudy,0,51.59,Autumn


In [59]:
columns_dict = {
  "Store ID" : "s_id",
  "Product ID" : "p_id",
  "Category" : "category",
  "Region" : "region",
  "Seasonality" : "seasonality",
  "Inventory Level" : "amount",
  "Units Sold" : "units_sold",
  "Units Ordered" : "units_ordered",
  "Price": "price",
  "Discount" : "discount"
}


drop_column = ["Date", "Demand Forecast", "Weather Condition", "Holiday/Promotion", "Competitor Pricing"]
df_dropped = df.drop(drop_column, axis=1)
df_dropped = df_dropped.rename(columns=columns_dict)

df1_columns = ['s_id', 'p_id', "category", "region", 'seasonality']
df2_columns = ['s_id', 'p_id', 'amount', "units_sold", "units_ordered", "price", "discount"]

def filter_columns (df: pd.DataFrame, preserved_cols: list) -> pd.DataFrame:
  cols = df.columns.tolist()
  for col in preserved_cols:
    cols.remove(col)
  
  return df.drop(cols, axis=1)

df1 = filter_columns(df_dropped, df1_columns)
df2 = filter_columns(df_dropped, df2_columns)

### Table 1 
Inventory Details

In [60]:
# Inventory details
# table name: inventory_details
df1.head()


,s_id,p_id,category,region,seasonality
0,S001,P0001,Groceries,North,Autumn
1,S001,P0002,Toys,South,Autumn
2,S001,P0003,Toys,West,Summer
3,S001,P0004,Toys,North,Autumn
4,S001,P0005,Electronics,East,Summer


In [61]:
df1['vt_start'] = None
df1['vt_end'] = None
df1['tt_start'] = None
df1['tt_end'] = None
df1.head()

,s_id,p_id,category,region,seasonality,vt_start,vt_end,tt_start,tt_end
0,S001,P0001,Groceries,North,Autumn,None,None,None,None
1,S001,P0002,Toys,South,Autumn,None,None,None,None
2,S001,P0003,Toys,West,Summer,None,None,None,None
3,S001,P0004,Toys,North,Autumn,None,None,None,None
4,S001,P0005,Electronics,East,Summer,None,None,None,None


In [62]:
from random import randrange
from datetime import timedelta
from datetime import datetime

def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

# d1 = datetime.strptime('1/1/2008 1:30 PM', '%m/%d/%Y %I:%M %p')
# d2 = datetime.strptime('31/1/2009 4:50 AM', '%m/%d/%Y %I:%M %p')

# print(random_date(d1, d2))

In [63]:
import random

def populate_df_details(df : pd.DataFrame, num_duplicate: int = 3):
  # Declare date
  min_date = datetime.strptime('1/1/2020 01:00 AM', '%m/%d/%Y %I:%M %p')
  now_date = datetime.now()

  result_data = []
  cols = df.columns.tolist()
  # Iterate for each data in df
  for _, row in df.iterrows():
    current_data = []
    num_duplicate_data = random.randint(0, num_duplicate)

    row_dict = row.to_dict()

    # Generate TT and VT
    vt_start = random_date(min_date, now_date)
    row_dict['vt_start'] = vt_start
    vt_end = random_date(vt_start, now_date)
    row_dict['vt_end'] = vt_end
    tt_start = random_date(min_date, now_date)
    row_dict['tt_start'] = tt_start
    tt_end = random_date(tt_start, now_date)
    row_dict['tt_end'] = tt_end
    current_data.append(row_dict)

    # print(row_dict)

    # Generate random data
    for _ in range(num_duplicate_data):
      cols_to_change = random.randint(2, len(row_dict)-4-1)
      col_name = cols[cols_to_change]
      unique_val = df[col_name].unique().tolist()
      # Choose a new value
      choice = random.choice(unique_val)
      while (choice == row_dict[col_name]):
        # Choose until the value is different
        choice = random.choice(unique_val)
      
      # print(f"[{col_name}] : {row_dict[col_name]} => {choice}")
      # Update tt
      new_tt_start = row_dict['tt_end']
      new_tt_end = random_date(new_tt_start, now_date)

      # Change Value
      new_row_dict = row_dict.copy()
      new_row_dict[col_name] = choice
      new_row_dict["tt_start"] = new_tt_start
      new_row_dict["tt_end"] = new_tt_end

      # Generate new VT (optional, 20% chance)
      vt_change_num = random.randint(1, 10)
      if (vt_change_num <= 2):
        new_vt_start = random_date(min_date, now_date)
        new_row_dict['vt_start'] = new_vt_start
        new_vt_end = random_date(new_vt_start, now_date)
        new_row_dict['vt_end'] = new_vt_end

      # print(row_dict)
      current_data.append(new_row_dict)

      # Update pointer
      row_dict = new_row_dict

    # Change last data vt_end and tt_end to be 'infinity'
    # Last data's tt_end should always be infinity
    current_data[-1]['tt_end'] = "infinity"
    current_data[-1]['vt_end'] = "infinity"

    # Extend 
    result_data.extend(current_data)

  # Change date time
  for data in result_data:
    data['vt_start'] = data['vt_start'].strftime('%Y-%m-%d %H:%M:%S')
    data['vt_end'] = "infinity" if data['vt_end'] == "infinity" else data['vt_end'].strftime('%Y-%m-%d %H:%M:%S')
    data['tt_start'] = data['tt_start'].strftime('%Y-%m-%d %H:%M:%S')
    data['tt_end'] = "infinity" if data['tt_end'] == "infinity" else data['tt_end'].strftime('%Y-%m-%d %H:%M:%S')
  return pd.DataFrame(result_data)


df1_populated = populate_df_details(df1)

In [64]:
df1_populated.to_csv(os.path.join(DATA_DIR, 'inventory_details.csv'), index=False)

### Table 2
Inventory Sales

In [65]:
# Inventory sales
# table name: inventory_sales
df2.head()

,s_id,p_id,amount,units_sold,units_ordered,price,discount
0,S001,P0001,231,127,55,33.50,20
1,S001,P0002,204,150,66,63.01,20
2,S001,P0003,102,65,51,27.99,10
3,S001,P0004,469,61,164,32.72,10
4,S001,P0005,166,14,135,73.64,0


In [66]:
df2['vt_start'] = None
df2['vt_end'] = None
df2['tt_start'] = None
df2['tt_end'] = None
df2.head()

,s_id,p_id,amount,units_sold,units_ordered,price,discount,vt_start,vt_end,tt_start,tt_end
0,S001,P0001,231,127,55,33.50,20,None,None,None,None
1,S001,P0002,204,150,66,63.01,20,None,None,None,None
2,S001,P0003,102,65,51,27.99,10,None,None,None,None
3,S001,P0004,469,61,164,32.72,10,None,None,None,None
4,S001,P0005,166,14,135,73.64,0,None,None,None,None


In [67]:
import random

def populate_df_sales(df : pd.DataFrame, num_duplicate: int = 20):
  # Declare date
  min_date = datetime.strptime('1/1/2020 01:00 AM', '%m/%d/%Y %I:%M %p')
  now_date = datetime.now()

  result_data = []
  cols = df.columns.tolist()
  # Iterate for each data in df
  for _, row in df.iterrows():
    current_data = []
    num_duplicate_data = random.randint(0, num_duplicate)

    row_dict = row.to_dict()

    # Generate TT and VT
    vt_start = random_date(min_date, now_date)
    row_dict['vt_start'] = vt_start
    vt_end = random_date(vt_start, now_date)
    row_dict['vt_end'] = vt_end
    tt_start = random_date(min_date, now_date)
    row_dict['tt_start'] = tt_start
    tt_end = random_date(tt_start, now_date)
    row_dict['tt_end'] = tt_end
    current_data.append(row_dict)

    # print(num_duplicate_data)
    # print(row_dict)

    # Generate random data
    for _ in range(num_duplicate_data):
      cols_to_change = random.randint(2, len(row_dict)-4-1)
      col_name = cols[cols_to_change]
      
      if (col_name in ['amount', 'units_sold']):
        num_change = random.randint(1, 60)
      elif (col_name == "units_ordered"):
        num_change = random.randint(1, 40)
      elif (col_name == "price"):
        num_change = random.uniform(1.00, 10.00)
        num_change = num_change if (random.randint(1, 2)) == 1 else -num_change
      else : # col_name == "discount"
        num_change = random.randint(1, 20)
        num_change = num_change if (random.randint(1, 2)) == 1 else -num_change

      
      # print(f"[{col_name}] : {row_dict[col_name]} => {num_change}")
      # Update tt
      new_tt_start = row_dict['tt_end']
      new_tt_end = random_date(new_tt_start, now_date)

      # Change Value
      new_row_dict = row_dict.copy()
      invalid = False
      if (col_name in ['amount', 'units_sold']):
        if (new_row_dict['amount'] == 0):
          invalid = True
        else:
          actual_num_change = min(num_change, new_row_dict['amount'])
          new_row_dict['amount'] -= actual_num_change
          new_row_dict['units_sold'] += actual_num_change
      elif (col_name == "units_ordered"):
        chance = random.randint(1,2)
        if (chance == 1): # add units_ordered
          new_row_dict['units_ordered'] += num_change
        else:
          if (new_row_dict['units_ordered'] == 0):
            invalid = True
          else:
            actual_num_change = min(num_change, new_row_dict['units_ordered'])
            new_row_dict['amount'] += actual_num_change
            new_row_dict['units_ordered'] -= actual_num_change
      elif (col_name == "price"):
        new_row_dict['price'] = round(new_row_dict['price'] + num_change, 2)
      else : # col_name == "discount"
        new_row_dict['discount'] += num_change

      # If invalid, then this iteration cannot be proceed
      if (not invalid):
        new_row_dict["tt_start"] = new_tt_start
        new_row_dict["tt_end"] = new_tt_end

        # Generate new VT (optional, 20% chance)
        vt_change_num = random.randint(1, 10)
        if (vt_change_num <= 2):
          new_vt_start = random_date(min_date, now_date)
          new_row_dict['vt_start'] = new_vt_start
          new_vt_end = random_date(new_vt_start, now_date)
          new_row_dict['vt_end'] = new_vt_end

        # print(new_row_dict)
        current_data.append(new_row_dict)

        # Update pointer
        row_dict = new_row_dict


    # Change last data vt_end and tt_end to be 'infinity'
    # Last data's tt_end should always be infinity
    current_data[-1]['tt_end'] = "infinity"
    current_data[-1]['vt_end'] = "infinity"

    # Extend 
    result_data.extend(current_data)

  # Change date time
  for data in result_data:
    data['vt_start'] = data['vt_start'].strftime('%Y-%m-%d %H:%M:%S')
    data['vt_end'] = "infinity" if data['vt_end'] == "infinity" else data['vt_end'].strftime('%Y-%m-%d %H:%M:%S')
    data['tt_start'] = data['tt_start'].strftime('%Y-%m-%d %H:%M:%S')
    data['tt_end'] = "infinity" if data['tt_end'] == "infinity" else data['tt_end'].strftime('%Y-%m-%d %H:%M:%S')
  
  # for d in result_data:
  #   print(d)

  return pd.DataFrame(result_data)


df2_populated = populate_df_sales(df2)

In [68]:
df2_populated.to_csv(os.path.join(DATA_DIR, 'inventory_sales.csv'), index=False)